In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import time
import xml.etree.ElementTree as ET


In [2]:

def initialize_driver():
    # Initialize Safari driver
    driver = webdriver.Safari()
    return driver


In [3]:
def fetch_video_links(driver, url):
    driver.get(url)
    # 等待网页加载并获取所有视频链接
    video_link_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.XPATH, "//*[@id='maincontent']//a[contains(@href, '/talks/')]"))
    )
    # 提取每个元素的 href 属性
    video_links = [element.get_attribute('href') for element in video_link_elements]
    return video_links


In [4]:

def accept_cookies(driver):
    try:
        accept_button = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="onetrust-accept-btn-handler"]'))
        )
        driver.execute_script("arguments[0].click();", accept_button)
    except Exception as e:
        print("No cookie button found or error clicking it:", e)


In [5]:

def pause_video(driver):
    try:
        # 等待视频开始播放（currentTime > 0）
        WebDriverWait(driver, 10).until(
            lambda driver: driver.execute_script("return document.getElementById('video').currentTime > 0;")
        )
        # 暂停视频
        driver.execute_script("document.getElementById('video').pause();")
    except Exception as e:
        print("Failed to pause the video:", e)



In [6]:

def open_transcript(driver):
    try:
        transcript_button = WebDriverWait(driver, 20).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="transcript-control"]'))
        ) 
        driver.execute_script("arguments[0].click();", transcript_button)
        time.sleep(5)
    except Exception as e:
        print("Error clicking on transcript button:", e)



In [7]:

def switch_language_js(driver, language_code):
    try:
        language_select = WebDriverWait(driver, 5).until(
            EC.element_to_be_clickable((By.XPATH, '/html/body/div[2]/div[1]/div[2]/div/div/div/div[2]/div[1]/div/select'))
        )
        select = Select(language_select)
        select.select_by_value(language_code)
        driver.execute_script("arguments[0].dispatchEvent(new Event('change'));", language_select)
        # 尝试额外触发input事件
        driver.execute_script("arguments[0].dispatchEvent(new Event('input'));", language_select)
        time.sleep(5)
    except Exception as e:
        print(f"Failed to set language with code '{language_code}':", e)


In [8]:

def save_transcript(driver, filename):
    time.sleep(5)  # 等待指定元素（字幕内容）加载完毕
    transcript_elements = WebDriverWait(driver, 30).until(
        EC.presence_of_all_elements_located((By.XPATH, '/html/body/div[2]/div[1]/div[2]/div/div/div/div[4]/div'))
    )
    root = ET.Element("transcript")
    for element in transcript_elements:
        ET.SubElement(root, "section", text=element.text)
    tree = ET.ElementTree(root)
    tree.write(filename)


In [10]:

    driver = initialize_driver()
    url = "https://www.ted.com/talks?sort=oldest&topics%5B0%5D=3d%20printing&topics%5B1%5D=ai&topics%5B2%5D=blockchain&topics%5B3%5D=computers&topics%5B4%5D=cyber%20security&topics%5B5%5D=data&topics%5B6%5D=driverless%20cars&topics%5B7%5D=drones&topics%5B8%5D=engineering&topics%5B9%5D=future&topics%5B10%5D=internet&topics%5B11%5D=nanotechnology&topics%5B12%5D=robots&topics%5B13%5D=social%20media&topics%5B14%5D=software&topics%5B15%5D=surveillance&topics%5B16%5D=technology&topics%5B17%5D=virtual%20reality"
    links = fetch_video_links(driver, url)
    accept_cookies(driver)
    video_number = 1  # 初始化视频编号
    for link in links:
        driver.get(link)
    
        pause_video(driver)
        open_transcript(driver)
        save_transcript(driver, f"english-{video_number}.xml")
        switch_language_js(driver, 'zh-cn')
        save_transcript(driver, f"chinese-{video_number}.xml")
        switch_language_js(driver, 'fr')
        save_transcript(driver, f"french-{video_number}.xml")
        video_number += 1  # 更新视频编号
    driver.quit()




Failed to set language with code 'zh-cn': Message: 

Failed to set language with code 'fr': Message: 

Error clicking on transcript button: Message: A node reference could not be resolved: Node with identifier 'node-4A9A14F4-FE82-4DAE-87DD-2D47377E6EFB' was not found; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception



TimeoutException: Message: 
